## Naming conventions :

In [8]:
# as we write python code then:
# 1- variables : lowercase seperated by underscore (my_variable)
# 2- methods : lowercase seperated by underscore (my_function)
# 3- constants : ALLCAPITAL

## Packages :

In [9]:
#import OS to share the project dir
import os
os.chdir('F:\\fourth_year\Pattern_Classification\project\Writer_Identification') 

#import important packages
from IAM_loader import IAM_loader
#import tensorflow as tf
#import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
import cv2 as cv2
from PIL import Image
import math
from skimage import feature

#packages for visualization
import matplotlib.pyplot as plt
import matplotlib.cm as cm
#allows charts to appear in the notebook 
%matplotlib inline

#ignoring warnings
import warnings
warnings.filterwarnings('ignore')

## Reading Data :

In [10]:
# we use IAM handwriting database download it first
# here we will read the images and get the important info about the images 
# we divide the available images into (train,cv and test)
# division will be depend on the number of the images 

training_data=[] # only the train 
training_data_threshold=[] #training black images 
validation_data=[] # cross validation 
test_data=[] # test images 

WIDTH=2175 #image width
HEIGHT=2304 #image height
path="../dataset/output2"

In [11]:
def reading_and_preprocess():
    loader=IAM_loader('../dataset')
    training_data,test_data,validation_data = loader.split_data()
    cv2.imwrite('../dataset/lines199.png',training_data[0][0])
    #writting the data 
    training_data_threshold=training_data
    for i in range(len(training_data)):
            #path_output = os.path.join(path,str(i)+'.png')
            temp=training_data[i][0]
            _,image= cv2.threshold(temp,0,255,cv2.THRESH_BINARY_INV)
            #cv2.imwrite(path_output,training_data[i][0])
            training_data_threshold[i][0]=image
    return training_data,test_data,validation_data,training_data_threshold

## Preprocessing Module:

### Crop the images:

In [12]:
#get the window that contains the words 
def crop_image(image,y1,y2,x1,x2):
    image=image[y1:y2,x1:x2]
    return image

### Resize Image:

In [13]:
#note : use WIDTH and HEIGHT 
def resize_image (img):
    img=cv2.resize(img, dsize=(HEIGHT, WIDTH), interpolation=cv2.INTER_CUBIC)
    return img

### Detect lines to get the region of handwritten document: 

In [14]:
def detect_lines(img):
    horizontal_lines=[]
    ret_lines=[] 
    #apply canny 
    edges = cv2.Canny(img,50,150,apertureSize = 3)
    #dilation
    kernel = np.ones((5,5), np.uint8)
    img_dilation = cv2.dilate(edges, kernel, iterations=1)
    #hough line transform
    lines = cv2.HoughLinesP(img_dilation, 1, np.pi/180,200, maxLineGap=1,minLineLength=img.shape[1]/2)
    #getting the horizontal lines 
    for i in range(len(lines)):
           for line in lines[i]:
                pt1 = (line[0],line[1])
                pt2 = (line[2],line[3])
                if (line[1]-line[3])==0 : #print((line[2]-line[3])//(line[0]-line[1]))
                        cv2.line(img, pt1, pt2, (255,255,255), 3) 
                        horizontal_lines.append(line)
    #sort lines to eleminate the closed lines 
    horizontal_lines=sorted(horizontal_lines,key=lambda x: x[1])
    pt=horizontal_lines[0]
    ret_lines.append(pt)
    #loop to retrive the included lines 
    for i in range(1,len(horizontal_lines)):
        if abs(horizontal_lines[i][1]-pt[1])>100:
            #print("included",horizontal_lines[i])
            pt=horizontal_lines[i]
            ret_lines.append(pt)
    return ret_lines

In [15]:
def get_region():
    for i in range(len(training_data)):
        img=training_data[i][0]
        lines=detect_lines(img)
        print(len(lines))
        print(img.shape,lines[1][1]+10,lines[2][1]-3,math.floor(lines[1][0]/4),img.shape[1]-1)
        cropped_img=crop_image(img,lines[1][1]+10,lines[2][1]-3,math.floor(lines[1][0]/4),img.shape[1]-1)
        cropped_img=resize_image(cropped_img)
        training_data[i][0]=cropped_img
        cv2.imwrite('../dataset/output/cropped'+str(i)+'.png',cropped_img)


### Getting Connected Comp:

In [16]:
def connected_comp(img):
    #ret, labels = cv2.connectedComponents(img)
    output=cv2.connectedComponentsWithStats(img, 4, cv2.CV_32S)
    ret=output[0]
    labels=output[1]
    stat=output[2]
    # Map component labels to hue val
    label_hue = np.uint8(179*labels/np.max(labels))
    blank_ch = 255*np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

    # cvt to BGR for display
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)

    # set bg label to black
    labeled_img[label_hue==0] = 0

    #cv2.imwrite('labeled.png', labeled_img)
    return ret,labels,stat,labeled_img

# ----------------------------------------------------------------------------------------

## Feature Engineering:

### SIFT :

In [38]:
def sift_fun (img):
    #img = cv.imread(img_path)
    #gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    kp = sift.detect(img,None)
    #img=cv.drawKeypoints(gray,kp,img)
    #cv.imwrite('sift_keypoints.jpg',img)
    return kp

### Texture Extraction :

In [18]:
def texture_extraction (labels,image,num_labels,stats):
    #define new image 
    output=np.zeros(shape=(image.shape))
    visited=np.zeros(shape=(labels.shape))
    h=10
    max_height_old=0
    max_height_new=0
    flag=0
    y_old=0
    avg_height=0
    num=0
    #loop in the rows and cols to get the labels 
    for i in range(labels.shape[0]):#height ->row by row
        max_height_new=0
        for j in range (labels.shape[1]):#widths -> col by col
            if visited[i][j]!=1 and labels[i][j]!=0:
                label=labels[i][j]
                # retrieving the width of the bounding box of the component
                width = stats[label, cv2.CC_STAT_WIDTH]
                # retrieving the height of the bounding box of the component
                height = stats[label, cv2.CC_STAT_HEIGHT]
                # retrieving the leftmost coordinate of the bounding box of the component
                x = stats[label, cv2.CC_STAT_LEFT] #x->col
                # retrieving the topmost coordinate of the bounding box of the component
                y = stats[label, cv2.CC_STAT_TOP] #y->row
                #rgb = cv2.rectangle(rgb, (x, y+height), (x+width, y), (255,160,122),1)
                if width>10:
                    #print("h=",h,"x=",x,"y=",y,"height=",height,"i=",i,"width=",width,i,"j=",j,"lbel=",label,"num=",num)
                    #copy the box of the label 
                    for k in range(h,h+height):
                        for l in range (x,x+width):
                            if image[y+k-h][l].all() ==0:
                                    continue
                            else :
                                 output[k][l]=image[y+k-h][l]
                            
                      #  output[h:h+height,x:x+width]=image[y:y+height,x:x+width]
                    visited[y:y+height,x:x+width]=1
                    if height>max_height_new:
                        max_height_new=height
                    #calculae avg height 
                    avg_height+=height
                    num+=1
                    
                #h=math.floor((h+5)/2)
        if flag==0 and max_height_new>0:
            max_height_old=max_height_new
            y_old=i
            flag=1
        elif num>0 and flag==1 and i>=(max_height_old+y_old):
            h=math.ceil(h+(avg_height/(2*num)))
            max_height_old=max_height_new
            y_old=i
            avg_height=0
            num=0
            
   
    return output,h+100         
    

In [19]:
def get_nine_texture(img):
    images=[]
    for i in range(9):
        images.append(img[i*128:(i+1)*127,i*256:(i+1)*255])
    return images

In [20]:
def get_texture():
    text_images=[]
    for i in range(len(training_data_threshold)):
        ret,labels,stats,image=connected_comp(training_data_threshold[i][0])
        output,h=texture_extraction(labels,training_data_threshold[i][0],ret,stats)
        print(i,h)
        img=crop_image(output,0,1100,0,output.shape[1])
        #cv2.imwrite("../dataset/output/texture"+str(i)+".png",img)
        images=get_nine_texture(img)
        text_images.append(images)
    return text_images

### LBP :


In [73]:
def LBP(text_images):
    hist_=[]
    labels=[]
    for i in range(len(training_data)):
        for j in range(9):
            # compute the Local Binary Pattern representation
            # of the image, and then use the LBP representation
            # to build the histogram of patterns
            lbp = feature.local_binary_pattern(text_images[i][j],8,1, method="edge") #num_of_pointts=24 radius=8
            (hist, _) = np.histogram(lbp.ravel(),
            bins=np.arange(0, 8+ 3),range=(0,8+ 2))
 
            # normalize the histogram
            hist = hist.astype("float")
            hist_.append(hist)
            labels.append(training_data[i][1])
    return hist_,labels
    

## Training Module:

### 1-Bayes Classifer:

In [22]:
def bayes_classifer():
    
    return 

### 2-SVM :

In [23]:
def SVM_fun(x,y,x_test):
    clf = svm.SVC(gamma=0.1,kernel='rbf')
    clf.fit(x, y) 
    y_test=clf.predict(x_test)
    return y_test

### 3-Linear/Logistic Regression:

In [24]:
def regression():
    return

### 4- K-NN:

In [25]:
def knn(X,y,test):
    neigh = KNeighborsClassifier(n_neighbors=3)
    neigh.fit(X, y) 
    labels_test=neigh.predict(test)
    return labels_test

### 5- Neural Network:

In [26]:
def neural_network():
    return

## Performance Module:

In [27]:
def get_accuracy():
    
    return

# Main Logic:

In [ ]:
#here we will call the previous methods in a certain sequence 
#in this step we read the data and doing some of the preprocessing 
#1-reading &splitting into training ,testing & CV
#2-binaraization using OTSU
#3-segmentation of the handwritten document
#4-writing output to a certain folder 
#5-apply inverse binarization
training_data,test_data,validation_data,training_data_threshold=reading_and_preprocess() 
get_region()
text_images=get_texture()

Adding document a01-000u to the training data
Adding document a01-000u to the training data
Adding document a01-000u to the training data
Adding document a01-000u to the training data
Adding document a01-000u to the training data
Adding document a01-000u to the training data
Adding document a01-000u to the training data
Adding document a01-000u to the training data
Adding document a01-000u to the training data
Adding document a01-000u to the training data
Adding document r06-035 to the training data
Adding document r06-035 to the training data
Adding document r06-035 to the training data
Adding document r06-035 to the training data
Adding document r06-035 to the training data
Adding document r06-035 to the training data
Adding document r06-035 to the training data
Adding document r06-035 to the training data
Adding document r06-035 to the training data
Adding document r06-035 to the training data
Adding document r06-090 to the training data
Adding document r06-090 to the training data


In [ ]:
hist,labels=LBP(text_images)
hist=np.array(hist)
#hist.reshape(len(hist),len((hist[0])))
count=0
labels_test=SVM_fun(hist,labels,hist)
for i in range(len(labels)):
    if labels[i]==labels_test[i]:
        count+=1
print("acc=",count/(len(labels)))


In [ ]:
labels_test=knn(hist,labels,hist)
count=0
for i in range(len(labels)):
    if labels[i]==labels_test[i]:
        count+=1
print("acc=",count/(len(labels)))

In [31]:
print(len(training_data))

30


In [39]:
X=[]
y=[]
for i in range(len(training_data)):
    img=sift_fun(training_data[i][0])
    X.append(img)
    y.append(trainin_data[i][1])
labels_test=knn(X,y,X)
count=0
for i in range(len(y)):
    if y[i]==labels_test[i]:
        count+=1
print("acc=",count/(len(y)))
print(labels_test)

error: OpenCV(3.4.3) C:\projects\opencv-python\opencv_contrib\modules\xfeatures2d\src\sift.cpp:1207: error: (-213:The function/feature is not implemented) This algorithm is patented and is excluded in this configuration; Set OPENCV_ENABLE_NONFREE CMake option and rebuild the library in function 'cv::xfeatures2d::SIFT::create'
